In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.conf import SparkConf
config = SparkConf()
#config.set("property", "value")
config.setMaster("local").setAppName("FileFormats")

from pyspark.sql import SparkSession
spark = SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()
sc=spark.sparkContext

In [ ]:
rawTickDfDf = spark.read.format("csv")\
                .option("header",True)\
                .option("inferSchema",True)\
                .load("hdfs://localhost:9000/layers/raw/csv/")
rawTickDfDf.printSchema()
rawTickDfDf.show(5)

In [ ]:
 from pyspark.sql.functions import col
    # .filter( (col("total_ratings") >= 100) & (col("avg_rating") >=3.5) )\
filterDf = rawTickDfDf.filter((col("price") > 0.0 )&(col( "volume" )> 0)).drop("HOUR","MIN")
#filterDf = rawTickDfDf.filter("(volume > 0)")
filterDf.show(5)

In [ ]:
filterDf.write.partitionBy("YEAR", "MONTH", "DAY","SYMBOL1").mode("overwrite").option("header", True)\
                .parquet("hdfs://localhost:9000/layers/raw/parquet1")